In [1]:

from pathlib import Path
import rasterio.mask

In [2]:
from reflectdetect.utils.orthophoto import load_panel_locations

path_all_false = "data/20240529_uav_multispectral_orthos_20m/orthophotos/IMG_0232_1.tif"
path_all_true = "data/20240529_uav_multispectral_orthos_20m/orthophotos/IMG_0104_1.tif"
panel_locations = load_panel_locations(Path("data/20240529_uav_multispectral_orthos_20m"), None)

panel_locations

C:\Users\ninja\miniconda3\envs\reflectdetect-test\lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on 'data\20240529_uav_multispectral_orthos_20m\panel_locations.gpkg'
  return ogr_read(
C:\Users\ninja\miniconda3\envs\reflectdetect-test\lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on 'data\20240529_uav_multispectral_orthos_20m\panel_locations.gpkg'
  return ogr_read(
C:\Users\ninja\miniconda3\envs\reflectdetect-test\lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on 'data\20240529_uav_multispectral_orthos_20m\panel_locations.gpkg'
  return ogr_read(


[('corner1blk',
                                     UUID   FID _description   latitude  \
  0  482be66a-f26f-49ff-a2b3-7a9143a105ee     1               51.564821   
  1  2f9a4bcc-8c30-4f5e-bbe7-35d58a30a55a     2               51.564812   
  2                                  None  None         None        NaN   
  3                                  None  None         None        NaN   
  
     longitude  elevation  ortho_ht                          time  fix_id  \
  0   9.917781     187.38       0.0  27-05-2024 11:14:10.000 MESZ     4.0   
  1   9.917758     187.40       0.0  27-05-2024 11:14:34.000 MESZ     4.0   
  2        NaN        NaN       NaN                          None     NaN   
  3        NaN        NaN       NaN                          None     NaN   
  
     h_accuracy  v_accuracy  speed  bearing                           geometry  
  0        0.01        0.01  0.011      0.0  POINT Z (9.91778 51.56482 187.38)  
  1        0.01        0.01  0.008      0.0   POINT Z (9

In [16]:
import matplotlib

matplotlib.use('Tkagg')

In [17]:
from rasterio.coords import BoundingBox
from shapely import Polygon
import numpy as np
import rasterio.plot
import matplotlib.pyplot as plt

for layer_name, location in panel_locations:
    panel = location
    if panel.empty:
        raise Exception("Invalid panel location, no corner points included")
    with rasterio.open(path_all_true) as orthophoto:
        bounds = BoundingBox(*orthophoto.bounds)
        panel_polygon = panel.union_all().convex_hull
        out_image, out_transform = rasterio.mask.mask(
            orthophoto, [panel_polygon], crop=True,
        )
        orthophoto_polygon = Polygon(
            [
                (bounds.left, bounds.bottom),
                (bounds.left, bounds.top),
                (bounds.right, bounds.top),
                (bounds.right, bounds.bottom),
            ]
        )
        for band in out_image:
            rasterio.plot.show(band)

        has_no_invalid_values = not np.any(out_image == 65535)

    # Check if all corner points of the panel are within the orthophoto bounds
    bool(panel.within(orthophoto_polygon).all()) and has_no_invalid_values
plt.show()